## DIA 051: Implementacion de un Sistema de Manejo de Errores en una API Flask

Hoy implementaremos un sistema robusto de manejo de errores en nuestra API Flask. Esto nos permitirá capturar excepciones, devolver respuestas estructuradas y mejorar la seguridad y depuración de nuestra aplicación.

El manejo adecuado de errores ayuda a:
✅ Evitar la exposición de información sensible.
✅ Ofrecer mensajes de error claros y detallados.
✅ Registrar errores en logs para facilitar la depuración.

🖥️ Código Completo (api.py)
python
Copiar
Editar
import os
import logging
from flask import Flask, jsonify, request
from flask_jwt_extended import JWTManager, create_access_token, jwt_required
from werkzeug.exceptions import HTTPException

# Configuración básica de la API
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'supersecretkey')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'jwtsecretkey')

# Inicialización de extensiones
jwt = JWTManager(app)

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Manejo de Errores Personalizado
# ---------------------------

class APIError(Exception):
    """Clase base para errores personalizados en la API."""
    def __init__(self, message, status_code=400):
        super().__init__()
        self.message = message
        self.status_code = status_code

    def to_dict(self):
        return {"error": self.message}

@app.errorhandler(APIError)
def handle_api_error(error):
    """Manejo de errores personalizados en la API."""
    response = jsonify(error.to_dict())
    response.status_code = error.status_code
    return response

@app.errorhandler(HTTPException)
def handle_http_exception(error):
    """Manejo de errores HTTP generales."""
    return jsonify({"error": error.description}), error.code

@app.errorhandler(Exception)
def handle_generic_exception(error):
    """Manejo de errores no controlados."""
    logger.error(f"Error inesperado: {error}")
    return jsonify({"error": "Ocurrió un error inesperado"}), 500

# ---------------------------
# Endpoints de la API
# ---------------------------

@app.route('/login', methods=['POST'])
def login():
    """
    Iniciar sesión y manejar errores de autenticación.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')

    if not username or not password:
        raise APIError("Faltan credenciales", 400)

    if username != "admin" or password != "1234":  # Simulación
        raise APIError("Credenciales inválidas", 401)

    token = create_access_token(identity=username)
    return jsonify(access_token=token), 200

@app.route('/protected', methods=['GET'])
@jwt_required()
def protected():
    """
    Endpoint protegido que requiere autenticación.
    """
    return jsonify({"msg": "Acceso permitido"}), 200

@app.route('/force_error', methods=['GET'])
def force_error():
    """
    Endpoint para probar el manejo de errores genéricos.
    """
    raise Exception("Este es un error intencional.")

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
🔍 Explicación de las Principales Implementaciones
🔹 Clase APIError para Errores Personalizados

Permite lanzar excepciones con mensajes personalizados.
Se usa en validaciones de login y otros endpoints.
🔹 Manejadores de Errores Globales
✅ @app.errorhandler(APIError): Captura excepciones de la clase APIError y devuelve una respuesta JSON estructurada.
✅ @app.errorhandler(HTTPException): Captura errores HTTP estándar como 404 Not Found.
✅ @app.errorhandler(Exception): Captura errores inesperados y los registra en logs.

🔹 Endpoints con Manejo de Errores
✅ /login: Valida credenciales y lanza errores personalizados si faltan datos o la autenticación falla.
✅ /protected: Un endpoint protegido que requiere autenticación JWT.
✅ /force_error: Endpoint de prueba que genera un error para verificar el manejo de excepciones.

